In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
from keras.callbacks import EarlyStopping

import time
from datetime import datetime

import pandas as pd
from numba import cuda
from datetime import datetime

In [2]:
def load_mnist_dataset(train_size=10000, flatten=True):
    """
    Load the MNIST data set consisting of handwritten single digit images 0-9
    formatted as 28 x 28 pixels with one channel (black and white)
    60000 images are for training
    10000 images are for testing
    This function reshapes the images as flattened 784 byte arrays, if needed
    The image bytes are normalized as float in [0,1]
    Y_train and Y_test are one-hot encoded for the 10 digit classes
    returns X_train, X_test, Y_train, Y_test
    """
    DIGIT_CLASSES = 10
    mnist = keras.datasets.mnist
    (X_train, Y_train), (X_test, Y_test) = mnist.load_data()

    #
    if flatten:
        #X_train is 60000 rows of 28x28 values --> reshaped in 60000 x 784
        RESHAPED = 784
        X_train = X_train.reshape(60000, RESHAPED)
        X_test = X_test.reshape(10000, RESHAPED)
    else:
        # Retain 2D image shape for CNN
        X_train = X_train.reshape((60000, 28, 28, 1))
        X_test = X_test.reshape((10000, 28, 28, 1))

            
    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')

    #normalize in [0,1]
    X_train /= 255
    X_test /= 255

    #one-hot
    Y_train = tf.keras.utils.to_categorical(Y_train, DIGIT_CLASSES)
    Y_test = tf.keras.utils.to_categorical(Y_test, DIGIT_CLASSES)
    ## Split off a smaller training sample of specified size and use
    ## the remainder of the training data as a validation set
    X_valid = X_train[train_size:]
    Y_valid = Y_train[train_size:]
    X_train = X_train[:train_size]
    Y_train = Y_train[:train_size]
    print(X_train.shape[0], 'train samples')
    print(X_valid.shape[0], 'validation samples')
    print(X_test.shape[0], 'test samples')
    
    return X_train, X_valid, X_test, Y_train, Y_valid, Y_test

In [3]:
X_train, X_valid, X_test, Y_train, Y_valid, Y_test = load_mnist_dataset(train_size=20000, flatten=False)

20000 train samples
40000 validation samples
10000 test samples


In [4]:
def create_CNN_Model1():
    DIGIT_CLASSES = 10
    IMG_ROWS, IMG_COLS = 28, 28 # input image dimensions
    INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 1)
    model = models.Sequential()
    # CONV => RELU => POOL
    model.add(layers.Convolution2D(20, (5, 5), activation='relu',
        input_shape=INPUT_SHAPE))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # CONV => RELU => POOL
    model.add(layers.Convolution2D(50, (5, 5), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # Flatten => RELU layers
    model.add(layers.Flatten())
    model.add(layers.Dense(500, activation='relu'))
    # a softmax classifier
    model.add(layers.Dense(DIGIT_CLASSES, activation="softmax"))
    return model
  
model1 = create_CNN_Model1()
model1.summary()
model1 = create_CNN_Model1

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 24, 24, 20)        520       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 12, 12, 20)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 8, 8, 50)          25050     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 50)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 800)               0         
                                                                 
 dense (Dense)               (None, 500)               4

In [5]:
def create_CNN_Model2():
    DIGIT_CLASSES = 10
    IMG_ROWS, IMG_COLS = 28, 28 # input image dimensions
    INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 1)
    model = models.Sequential()
    # CONV => RELU => POOL
    model.add(layers.Convolution2D(40, (3, 3), activation='relu', padding='same',
        input_shape=INPUT_SHAPE))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # CONV => RELU => POOL
    model.add(layers.Convolution2D(50, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # Flatten => RELU layers
    model.add(layers.Flatten())
    model.add(layers.Dense(500, activation='relu'))
    # a softmax classifier
    model.add(layers.Dense(DIGIT_CLASSES, activation="softmax"))
    return model

model2 = create_CNN_Model2()
model2.summary()
model2 = create_CNN_Model2

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 28, 28, 40)        400       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 14, 14, 40)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 14, 14, 50)        18050     
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 7, 7, 50)         0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 2450)              0         
                                                                 
 dense_2 (Dense)             (None, 500)              

In [6]:
def create_CNN_Model3():
    DIGIT_CLASSES = 10
    IMG_ROWS, IMG_COLS = 28, 28 # input image dimensions
    INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 1)
    model = models.Sequential()
    # CONV => RELU => POOL
    model.add(layers.Convolution2D(40, (3, 3), activation='relu', padding='same',
        input_shape=INPUT_SHAPE))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # CONV => RELU => POOL
    model.add(layers.Convolution2D(50, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # CONV => RELU
    model.add(layers.Convolution2D(50, (3, 3), activation='relu', padding='valid'))
    # Flatten => RELU layers
    model.add(layers.Flatten())
    model.add(layers.Dense(500, activation='relu'))
    # a softmax classifier
    model.add(layers.Dense(DIGIT_CLASSES, activation="softmax"))
    return model

model3 = create_CNN_Model3()
model3.summary()
model3 = create_CNN_Model3

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 28, 28, 40)        400       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 14, 14, 40)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 14, 14, 50)        18050     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 7, 7, 50)         0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 5, 5, 50)          22550     
                                                                 
 flatten_2 (Flatten)         (None, 1250)             

In [7]:
def create_CNN_Model4():
    DIGIT_CLASSES = 10
    IMG_ROWS, IMG_COLS = 28, 28 # input image dimensions
    INPUT_SHAPE = (IMG_ROWS, IMG_COLS, 1)
    model = models.Sequential()
    # CONV => RELU => CONV => RELU => POOL
    model.add(layers.Convolution2D(20, (3,3), activation='relu',
        input_shape=INPUT_SHAPE))
    model.add(layers.Convolution2D(50, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # CONV => RELU => CONV => RELU => POOL
    model.add(layers.Convolution2D(50, (3,3), activation='relu'))
    model.add(layers.Convolution2D(50, (3,3), activation='relu'))
    model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    # Flatten => RELU layers
    model.add(layers.Flatten())
    model.add(layers.Dense(500, activation='relu'))
    # a softmax classifier
    model.add(layers.Dense(DIGIT_CLASSES, activation="softmax"))
    return model
  
model4 = create_CNN_Model4()
model4.summary()
model4 = create_CNN_Model4


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 26, 26, 20)        200       
                                                                 
 conv2d_8 (Conv2D)           (None, 24, 24, 50)        9050      
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 12, 12, 50)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 10, 10, 50)        22550     
                                                                 
 conv2d_10 (Conv2D)          (None, 8, 8, 50)          22550     
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 4, 4, 50)         0         
 2D)                                                  

In [8]:
def run_CNN_Model(model_name=None, model_fn=None, train_size=10000, verbose=0):
    OPTIMIZER = tf.keras.optimizers.Adam()
    BATCH_SIZE = 128
    EPOCHS = 200
    X_train, X_valid, X_test, Y_train, Y_valid, Y_test = load_mnist_dataset(train_size=train_size, flatten=False)
    
    ## invoke the passed in model creation function
    model = model_fn()
    
    model.compile(loss="categorical_crossentropy", optimizer=OPTIMIZER,
    metrics=["accuracy"])


    early_stopping = EarlyStopping(monitor='val_loss',verbose=1,patience=3,min_delta=0.00001)

    callbacks = [
        early_stopping,
    ]
    start_time = time.time()
    # fit 
    history = model.fit(X_train, Y_train,
                        batch_size=BATCH_SIZE, epochs=EPOCHS,
                        verbose=verbose, validation_data=(X_valid,Y_valid),
                        callbacks=callbacks)
    elapsed_time = time.time() - start_time

    test_loss, test_accuracy = model.evaluate(X_test, Y_test, verbose=1)
    print(f'training elapsed_time = {elapsed_time:10.1f}')
    print(f'{test_loss=}')
    print(f'{test_accuracy=}')
    return [model_name, train_size, elapsed_time, test_accuracy]


In [9]:
model_set = [('model1',model1),
             ('model2',model2),
             ('model3',model3),
             ('model4',model4),
            ]

In [10]:
def run_model_set(model_set, repeat=1):
    now = datetime.now()
    result_name = now.strftime("data/CNN_DATA_%Y-%m-%d_%H_%M.csv")
    results = []
    for i in range(repeat):
        for model_name, model in model_set:
            result = run_CNN_Model(model_name=model_name, model_fn=model, train_size=50000)
            results.append(result)
        df = pd.DataFrame(results)
        df.columns=['model_name','train_size','training_time','test_accuracy']
        display(df)
        print(f'Saving results to file {result_name}')
        df.to_csv(result_name,index=False)
    return results

## results = run_model_set(model_set, repeat=20)


# df = pd.DataFrame(results)
# df.columns=['model_name','train_size','training_time','test_accuracy']
# display(df)
# now = datetime.now()
# result_name = now.strftime("data/CNN_DATA_%Y-%m-%d_%H:%M.csv")
# print(f'Saving results to file {result_name}')

# df.to_csv(result_name,index=False)


In [11]:
# df = pd.DataFrame(results)
# df.columns=['model_name','train_size','training_time','test_accuracy']
# display(df)
# now = datetime.now()
# result_name = now.strftime("data/CNN_DATA_%Y-%m-%d_%H_%M.csv")
# print(f'Saving results to file {result_name}')

# df.to_csv(result_name,index=False)


In [12]:
df = pd.read_csv('data/CNN_DATA_2022-09-28_21_18.csv')
df

,model_name,train_size,training_time,test_accuracy
0,model1,50000,30.376704,0.9902
1,model2,50000,42.672199,0.9878
2,model3,50000,29.635418,0.9904
3,model4,50000,65.077671,0.9924
4,model1,50000,24.425643,0.9898
...,...,...,...,...
75,model4,50000,70.128883,0.9926
76,model1,50000,33.541008,0.9917
77,model2,50000,42.358860,0.9882
78,model3,50000,38.853625,0.9912


In [ ]:
train_sizes = [50000, 20000, 10000, 5000, 2500, 1200, 600, 300]

def run_model_set_size(train_sizes=[50000], model_set=None, repeat=1):
    now = datetime.now()
    result_name = now.strftime("data/CNN_DATA_%Y-%m-%d_%H_%M.csv")
    results = []
    for train_size in train_sizes:
        for i in range(repeat):
            for model_name, model in model_set:
                result = run_CNN_Model(model_name=model_name, model_fn=model, train_size=train_size)
                results.append(result)
            df = pd.DataFrame(results)
            df.columns=['model_name','train_size','training_time','test_accuracy']
            display(df)
            print(f'Saving results to file {result_name}')
            df.to_csv(result_name,index=False)
    return results

results = run_model_set_size(train_sizes, model_set = model_set, repeat=4)

50000 train samples
10000 validation samples
10000 test samples
Epoch 7: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.0279 - accuracy: 0.9917
training elapsed_time =       24.0
test_loss=0.02791009284555912
test_accuracy=0.9916999936103821
50000 train samples
10000 validation samples
10000 test samples
Epoch 11: early stopping
313/313 [==============================] - 1s 4ms/step - loss: 0.0414 - accuracy: 0.9893
training elapsed_time =       49.1
test_loss=0.04141027107834816
test_accuracy=0.989300012588501
50000 train samples
10000 validation samples
10000 test samples
Epoch 10: early stopping
313/313 [==============================] - 1s 4ms/step - loss: 0.0295 - accuracy: 0.9917
training elapsed_time =       46.4
test_loss=0.029500091448426247
test_accuracy=0.9916999936103821
50000 train samples
10000 validation samples
10000 test samples
Epoch 8: early stopping
313/313 [==============================] - 1s 4ms/step - loss: 0.0284 - accuracy: 0.9

,model_name,train_size,training_time,test_accuracy
0,model1,50000,23.973921,0.9917
1,model2,50000,49.130800,0.9893
2,model3,50000,46.421665,0.9917
3,model4,50000,52.340361,0.9922


Saving results to file data/CNN_DATA_2022-09-29_09_22.csv
50000 train samples
10000 validation samples
10000 test samples
Epoch 8: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.0333 - accuracy: 0.9903
training elapsed_time =       24.6
test_loss=0.0333203487098217
test_accuracy=0.9902999997138977
50000 train samples
10000 validation samples
10000 test samples
Epoch 7: early stopping
313/313 [==============================] - 1s 4ms/step - loss: 0.0374 - accuracy: 0.9883
training elapsed_time =       31.6
test_loss=0.03736558556556702
test_accuracy=0.9883000254631042
50000 train samples
10000 validation samples
10000 test samples
Epoch 8: early stopping
313/313 [==============================] - 1s 4ms/step - loss: 0.0351 - accuracy: 0.9887
training elapsed_time =       37.3
test_loss=0.035052597522735596
test_accuracy=0.9886999726295471
50000 train samples
10000 validation samples
10000 test samples
Epoch 8: early stopping
313/313 [====================

,model_name,train_size,training_time,test_accuracy
0,model1,50000,23.973921,0.9917
1,model2,50000,49.130800,0.9893
2,model3,50000,46.421665,0.9917
3,model4,50000,52.340361,0.9922
4,model1,50000,24.637258,0.9903
5,model2,50000,31.619771,0.9883
6,model3,50000,37.253684,0.9887
7,model4,50000,51.389733,0.9927


Saving results to file data/CNN_DATA_2022-09-29_09_22.csv
50000 train samples
10000 validation samples
10000 test samples
Epoch 9: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.0430 - accuracy: 0.9886
training elapsed_time =       27.4
test_loss=0.0429900623857975
test_accuracy=0.9886000156402588
50000 train samples
10000 validation samples
10000 test samples
Epoch 7: early stopping
313/313 [==============================] - 1s 4ms/step - loss: 0.0282 - accuracy: 0.9910
training elapsed_time =       31.3
test_loss=0.028223074972629547
test_accuracy=0.9909999966621399
50000 train samples
10000 validation samples
10000 test samples
Epoch 9: early stopping
313/313 [==============================] - 1s 4ms/step - loss: 0.0329 - accuracy: 0.9918
training elapsed_time =       41.7
test_loss=0.032853301614522934
test_accuracy=0.9918000102043152
50000 train samples
10000 validation samples
10000 test samples
Epoch 6: early stopping
313/313 [===================

...

Saving results to file data/CNN_DATA_2022-09-29_09_22.csv
600 train samples
59400 validation samples
10000 test samples
Epoch 22: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.2744 - accuracy: 0.9235
training elapsed_time =       39.5
test_loss=0.27442193031311035
test_accuracy=0.9235000014305115
600 train samples
59400 validation samples
10000 test samples
Epoch 16: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.3262 - accuracy: 0.9112
training elapsed_time =       35.5
test_loss=0.3262273073196411
test_accuracy=0.9111999869346619
600 train samples
59400 validation samples
10000 test samples
Epoch 13: early stopping
313/313 [==============================] - 2s 5ms/step - loss: 0.3134 - accuracy: 0.9099
training elapsed_time =       30.7
test_loss=0.3134007453918457
test_accuracy=0.9099000096321106
600 train samples
59400 validation samples
10000 test samples
Epoch 12: early stopping
313/313 [==========================

,model_name,train_size,training_time,test_accuracy
0,model1,50000,23.973921,0.9917
1,model2,50000,49.130800,0.9893
2,model3,50000,46.421665,0.9917
3,model4,50000,52.340361,0.9922
4,model1,50000,24.637258,0.9903
...,...,...,...,...
95,model4,1200,33.517739,0.9350
96,model1,600,39.467771,0.9235
97,model2,600,35.460911,0.9112
98,model3,600,30.674804,0.9099


Saving results to file data/CNN_DATA_2022-09-29_09_22.csv
600 train samples
59400 validation samples
10000 test samples
Epoch 19: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.2851 - accuracy: 0.9180
training elapsed_time =       35.5
test_loss=0.2851150333881378
test_accuracy=0.9179999828338623
600 train samples
59400 validation samples
10000 test samples
Epoch 17: early stopping
313/313 [==============================] - 1s 4ms/step - loss: 0.3314 - accuracy: 0.9072
training elapsed_time =       38.6
test_loss=0.33144327998161316
test_accuracy=0.9071999788284302
600 train samples
59400 validation samples
10000 test samples
Epoch 14: early stopping
313/313 [==============================] - 1s 4ms/step - loss: 0.3322 - accuracy: 0.9047
training elapsed_time =       32.5
test_loss=0.332233190536499
test_accuracy=0.904699981212616
600 train samples
59400 validation samples
10000 test samples
Epoch 13: early stopping
313/313 [============================

,model_name,train_size,training_time,test_accuracy
0,model1,50000,23.973921,0.9917
1,model2,50000,49.130800,0.9893
2,model3,50000,46.421665,0.9917
3,model4,50000,52.340361,0.9922
4,model1,50000,24.637258,0.9903
...,...,...,...,...
99,model4,600,37.323063,0.9066
100,model1,600,35.501542,0.9180
101,model2,600,38.647165,0.9072
102,model3,600,32.460921,0.9047


Saving results to file data/CNN_DATA_2022-09-29_09_22.csv
600 train samples
59400 validation samples
10000 test samples
Epoch 20: early stopping
313/313 [==============================] - 1s 4ms/step - loss: 0.2915 - accuracy: 0.9174
training elapsed_time =       36.6
test_loss=0.2915057837963104
test_accuracy=0.9174000024795532
600 train samples
59400 validation samples
10000 test samples
Epoch 14: early stopping
313/313 [==============================] - 1s 3ms/step - loss: 0.3534 - accuracy: 0.8993
training elapsed_time =       31.3
test_loss=0.35337480902671814
test_accuracy=0.8992999792098999
600 train samples
59400 validation samples
10000 test samples
Epoch 13: early stopping
313/313 [==============================] - 1s 4ms/step - loss: 0.3297 - accuracy: 0.9022
training elapsed_time =       33.6
test_loss=0.32965582609176636
test_accuracy=0.9021999835968018
600 train samples
59400 validation samples
10000 test samples


In [ ]:
df = pd.DataFrame(results)
df.columns=['model_name','train_size','training_time','test_accuracy']
display(df)
